# Рекомендация тарифов

**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.


## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data['calls'] = data['calls'].astype(int)
data['messages'] = data['messages'].astype(int)

In [6]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
data.duplicated().sum()

0

**Вывод по 1 пункту**: 
- в таблице нет пропусков;
- в столбцах **calls**, **messages** изменила тип данных;
- явных дубликатов нет.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Хороший первичный анализ данных

## Разбейте данные на выборки

Разобьем сначала на два набора: признаки (**features**) и целевой признак (**target**).

In [8]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

Далее разобьем каждый набор на обучащую, валидационную и тестовую выборку в соотношении 60:20:20.

In [9]:
features_train,features_valid_test,target_train, target_valid_test = train_test_split(features, target, test_size=0.4, random_state=12345)

In [10]:
features_test, features_valid, target_test, target_valid= train_test_split(features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

In [11]:
print('Исходная выборка:','размер features -', features.shape,',', 'размер target -', target.shape)
print('Обучающая выборка:','размер features-', features_train.shape,',', 'размер target -', target_train.shape)
print('Валидационная выборка:', 'размер features-', features_valid.shape,',', 'размер target-', target_valid.shape)
print('Тестовая выборка:','размер features-', features_test.shape,',', 'размер target-', target_test.shape)


Исходная выборка: размер features - (3214, 4) , размер target - (3214,)
Обучающая выборка: размер features- (1928, 4) , размер target - (1928,)
Валидационная выборка: размер features- (643, 4) , размер target- (643,)
Тестовая выборка: размер features- (643, 4) , размер target- (643,)


<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Выборки разделены верно, здорово, что проверены их размеры

## Исследуйте модели

Расcмотрим три модели с различными гиперпараметрами: 
   - DecisionTreeClassifier;
   - DecisionTreeClassifier;
   - LogisticRegression.

**DecisionTreeClassifier**

In [12]:
best_depth_decision_tree_classifier = 0
best_accuracy_decision_tree_classifier = 0.75
best_split = 0
best_criterion = None
list_criterion = ['gini', 'entropy']

for crit in list_criterion:
    for split in range(2,10):
        for depth in range(1, 11):
            model_decision_tree_classifier = DecisionTreeClassifier(criterion=crit,min_samples_split=split, random_state=12345, max_depth = depth)
            model_decision_tree_classifier.fit(features_train, target_train);
            prediction_decision_tree_classifier = model_decision_tree_classifier.predict(features_valid)
            accuracy_decision_tree_classifier = accuracy_score(target_valid, prediction_decision_tree_classifier)
    
            if accuracy_decision_tree_classifier > best_accuracy_decision_tree_classifier:
                model_best_decision_tree_classifier = model_decision_tree_classifier
                best_accuracy_decision_tree_classifier = accuracy_decision_tree_classifier;
                best_depth_decision_tree_classifier = depth
                best_criterion = crit
                best_split = split

print(f'Accuracy лучшей модели на валидационной выборке: {best_accuracy_decision_tree_classifier}')
print(f'Глубина лучшей модели на валидационной выборке: {best_depth_decision_tree_classifier}')
print(f'Лучшая функция качества модели на валидационной выборке: {best_criterion}')
print(f'Лучшее минимальное количество примеров для разделения: {best_split}')
    

Accuracy лучшей модели на валидационной выборке: 0.8009331259720062
Глубина лучшей модели на валидационной выборке: 8
Лучшая функция качества модели на валидационной выборке: entropy
Лучшее минимальное количество примеров для разделения: 2


**RandomForestClassifier**

In [13]:
best_accuracy_random_forest_classifier = 0.75
best_est_random_forest_classifier = 0
best_depth_random_forest_classifier = 0
best_max_features_random_forest_classifier = None
best_crit_random_forest_classifier = None
list_max_features = ['sqrt', 'log2']    

for mf in list_max_features:
    for crit in list_criterion:
        for est in range(1,20):
    
            for depth in range(1,11):
                model_random_forest_classifier = RandomForestClassifier(criterion=crit, random_state=12345, n_estimators=est, max_depth=depth, max_features = mf) 
                model_random_forest_classifier.fit(features_train, target_train);
                prediction_random_forest_classifier = model_random_forest_classifier.predict(features_valid)
                accuracy_random_forest_classifier = accuracy_score(target_valid,prediction_random_forest_classifier)
        
                if accuracy_random_forest_classifier >= best_accuracy_random_forest_classifier:
                    model_best_random_forest_classifier = model_random_forest_classifier
                    best_accuracy_random_forest_classifier = accuracy_random_forest_classifier;
                    best_est_random_forest_classifier = est
                    best_depth_random_forest_classifier = depth
                    best_crit_random_forest_classifier = crit
                    best_max_features_random_forest_classifier = mf

            
print(f'Accuracy лучшей модели на валидационной выборке: {best_accuracy_random_forest_classifier}')
print(f'Количество деревьев лучшей модели на валидационной выборке: {best_est_random_forest_classifier}')
print(f'Глубина лучшей модели на валидационной выборке: {best_depth_random_forest_classifier}')
print(f'Лучшая функция качества модели на валидационной выборке: {best_crit_random_forest_classifier}')
print(f'Лучшее число признаков для выбора расщепления: {best_max_features_random_forest_classifier}')

Accuracy лучшей модели на валидационной выборке: 0.8242612752721618
Количество деревьев лучшей модели на валидационной выборке: 17
Глубина лучшей модели на валидационной выборке: 9
Лучшая функция качества модели на валидационной выборке: entropy
Лучшее число признаков для выбора расщепления: log2


**LogisticRegression**

In [23]:
solvers = ['lbfgs', 'liblinear', 'newton-cg',  'sag'];
best_accuracy_lr = 0
best_solver=None
best_lr_model= None
for solv in solvers:
    model_logistic_regression = LogisticRegression( random_state=12345, solver=solv, max_iter=1000);
    model_logistic_regression.fit(features_train, target_train);
    prediction_logistic_regression = model_logistic_regression.predict(features_valid)
    accuracy_prediction_logistic_regression = accuracy_score(target_valid,prediction_logistic_regression)
    if accuracy_prediction_logistic_regression > best_accuracy_lr:
        best_lr_model =  model_logistic_regression
        best_solver = solv
        best_accuracy_lr = accuracy_prediction_logistic_regression
           
print(f'Accuracy лучшей модели на валидационной выборке: {accuracy_prediction_logistic_regression}')
print(f'Лучшей алгоритм для оптимизиции: {best_solver}')


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorith

Accuracy лучшей модели на валидационной выборке: 0.6842923794712286
Лучшей алгоритм для оптимизиции: newton-cg


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


**Вывод по 3 пункту**: 
- самое высокое качество **accuracy = 0.824261** обучения показала модель **RandomForestClassifier** c гипперпараметрами: **max_depth = 9** (глубина дерева),  **n_estimators = 17** (количестово деревьев), **criterion = entropy**, **max_features** = log2
- самое низкое качество показала модель **LogisticRegression**.

## Проверьте модель на тестовой выборке

In [15]:
accuracy_tree_test = model_best_decision_tree_classifier.score(features_test, target_test)
print(f'Accuracy  DecisionTreeClassifier на тестовой выборке: {accuracy_tree_test}')

Accuracy  DecisionTreeClassifier на тестовой выборке: 0.7776049766718507


In [16]:
accuracy_forest_test = model_best_random_forest_classifier.score(features_test, target_test)
print(f'Accuracy DecisionTreeClassifier на тестовой выборке: {accuracy_forest_test}')

Accuracy DecisionTreeClassifier на тестовой выборке: 0.8055987558320373


In [17]:
accuracy_logistic_test = model_logistic_regression.score(features_test, target_test)
print(f'Accuracy DecisionTreeClassifier на тестовой выборке: {accuracy_logistic_test}')

Accuracy DecisionTreeClassifier на тестовой выборке: 0.7045101088646968


In [18]:
accuracy = max(accuracy_tree_test,accuracy_forest_test, accuracy_logistic_test)

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Модели проверены на тесте

## (бонус) Проверьте модели на адекватность

In [19]:
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(features_train, target_train)
dummy_valid_accuracy = dummy_model.score(features_valid, target_valid)
dummy_test_accuracy = dummy_model.score(features_test, target_test)
print(f'Accuracy DummyClassifier: {dummy_test_accuracy}')
print(f'Accuracy нашего прогноза {accuracy}')

Accuracy DummyClassifier: 0.7060653188180405
Accuracy нашего прогноза 0.8055987558320373


In [20]:
dummy_model = DummyClassifier(strategy="uniform")
dummy_model.fit(features_train, target_train)
dummy_valid_accuracy = dummy_model.score(features_valid, target_valid)
dummy_test_accuracy = dummy_model.score(features_test, target_test)
print(f'Accuracy DummyClassifier: {dummy_test_accuracy}')
print(f'Accuracy нашего прогноза {accuracy}')

Accuracy DummyClassifier: 0.47900466562986005
Accuracy нашего прогноза 0.8055987558320373


In [21]:
dummy_model = DummyClassifier(strategy="stratified")
dummy_model.fit(features_train, target_train)
dummy_valid_accuracy = dummy_model.score(features_valid, target_valid)
dummy_test_accuracy = dummy_model.score(features_test, target_test)
print(f'Accuracy DummyClassifier: {dummy_test_accuracy}')
print(f'Accuracy нашего прогноза {accuracy}')

Accuracy DummyClassifier: 0.5443234836702955
Accuracy нашего прогноза 0.8055987558320373


## Вывод
Было проверено три модели: 
- **DecisionTreeClassifier**(Дерево решений), 
- **RandomForestClassifier** (Случайный лес), 
- **LogisticRegression**(Логическая регрессия). 

Cамый высокий показатель **accuracy = 0.824261** обучения показала модель **RandomForestClassifier** c гипперпараметрами: **max_depth = 9** (глубина дерева),  **n_estimators = 17** (количестово деревьев), **criterion = entropy** (функция измерения качества разделения), **max_features** = log2 (число признаков для выбора расщепления).